# **Data Wrangling**

## 1. Load the dataset

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("data/survey.csv")

## 2. Data Wrangling

### 2.1. Duplicates

In [42]:
df.duplicated().sum()

154

In [43]:
df['Respondent'].duplicated().sum()

154

### 2.2. Removing Duplicates

In [44]:
df.drop_duplicates(inplace=True)

In [45]:
df.duplicated().sum()

0

In [46]:
print(len(df.index))

11398


In [47]:
print(len(df['Respondent'].unique()))

11398


### 2.3. Missing Values

In [48]:
missing_data = df.isnull()

for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

Respondent
Respondent
False    11398
Name: count, dtype: int64

MainBranch
MainBranch
False    11398
Name: count, dtype: int64

Hobbyist
Hobbyist
False    11398
Name: count, dtype: int64

OpenSourcer
OpenSourcer
False    11398
Name: count, dtype: int64

OpenSource
OpenSource
False    11317
True        81
Name: count, dtype: int64

Employment
Employment
False    11398
Name: count, dtype: int64

Country
Country
False    11398
Name: count, dtype: int64

Student
Student
False    11347
True        51
Name: count, dtype: int64

EdLevel
EdLevel
False    11286
True       112
Name: count, dtype: int64

UndergradMajor
UndergradMajor
False    10661
True       737
Name: count, dtype: int64

EduOther
EduOther
False    11234
True       164
Name: count, dtype: int64

OrgSize
OrgSize
False    11302
True        96
Name: count, dtype: int64

DevType
DevType
False    11333
True        65
Name: count, dtype: int64

YearsCode
YearsCode
False    11389
True         9
Name: count, dtype: int64

Age1stCode
Age

In [49]:
missing_values_count = df.isnull().sum()

top_5_columns_with_most_missing = missing_values_count.sort_values(ascending=False).head(3)

print("3 Columns with the most missing values:")
for column, missing_count in top_5_columns_with_most_missing.items():
    print(f"Col {column}: {missing_count} missing values ({round(100*missing_count/len(df), 2)}%)")

3 Columns with the most missing values:
Col BlockchainIs: 2610 missing values (22.9%)
Col CodeRevHrs: 2426 missing values (21.28%)
Col BlockchainOrg: 2322 missing values (20.37%)


Some overlooks about some main columns

### 2.4. Imputing Missing Values

Find the  value counts for the column WorkLoc.

In [50]:
df['BlockchainIs'].value_counts()

BlockchainIs
Useful for immutable record keeping outside of currency                  2601
Useful across many domains and could change many aspects of our lives    2425
A passing fad                                                            1521
An irresponsible use of resources                                        1341
Useful for decentralized currency (i.e., Bitcoin)                         900
Name: count, dtype: int64

In [51]:
df['CodeRevHrs'].value_counts().head(5)

CodeRevHrs
2.0    1768
5.0    1657
4.0    1187
3.0    1117
1.0     989
Name: count, dtype: int64

In [52]:
df['BlockchainOrg'].value_counts()

BlockchainOrg
Not at all                                                 7215
Non-currency applications of blockchain                    1209
Implementing cryptocurrency-based products                  384
Accepting Bitcoin or other coins and tokens as payments     152
Implementing our own cryptocurrency                         116
Name: count, dtype: int64

Thus, we cannot imputing missing value because in each column, no category dominates others. Just leave it as null

### 2.5. Normalizing Data

There are two columns in the dataset that talk about compensation.

One is "CompFreq". This column shows how often a developer is paid (Yearly, Monthly, Weekly).

The other is "CompTotal". This column talks about how much the developer is paid per Year, Month, or Week depending upon his/her "CompFreq".

This makes it difficult to compare the total compensation of the developers.

In this section you will create a new column called 'NormalizedAnnualCompensation' which contains the 'Annual Compensation' irrespective of the 'CompFreq'.

Once this column is ready, it makes comparison of salaries easy.

<hr>

In [53]:
df['CompFreq'].unique()

array(['Yearly', 'Monthly', 'Weekly', nan], dtype=object)

In [54]:
df['CompFreq'].value_counts()

CompFreq
Yearly     6073
Monthly    4788
Weekly      331
Name: count, dtype: int64

Use the below logic to arrive at the values for the column NormalizedAnnualCompensation.

- If the CompFreq is Yearly then use the exising value in CompTotal
- If the CompFreq is Monthly then multiply the value in CompTotal with 12 (months in an year)
- If the CompFreq is Weekly then multiply the value in CompTotal with 52 (weeks in an year)

In [55]:
def conditions(s):
  if (s['CompFreq'] == 'Yearly'):
    return s['CompTotal']
  elif (s['CompFreq'] == 'Monthly'):
    return (s['CompTotal'] * 12)  
  else:
    return (s['CompTotal'] * 52)

df['NormalizedAnnualCompensation'] = df.apply(conditions, axis=1)
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase,NormalizedAnnualCompensation
0,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy,61000.0
1,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,NaN,23.0,Man,No,Bisexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult,138000.0
2,13,I am a developer by profession,Yes,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Tech articles written by other developers;Cour...,28.0,Man,No,Straight / Heterosexual,White or of European descent,Yes,Appropriate in length,Easy,90000.0
3,16,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,Employed full-time,United Kingdom,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",NaN,...,Tech articles written by other developers;Indu...,26.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult,348000.0
4,17,I am a developer by profession,Yes,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,Employed full-time,Australia,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Tech articles written by other developers;Indu...,29.0,Man,No,Straight / Heterosexual,Hispanic or Latino/Latina;Multiracial,No,Appropriate in length,Easy,90000.0


In [56]:
df[['CompFreq', 'CompTotal', 'NormalizedAnnualCompensation']].head()

,CompFreq,CompTotal,NormalizedAnnualCompensation
0,Yearly,61000.0,61000.0
1,Yearly,138000.0,138000.0
2,Yearly,90000.0,90000.0
3,Monthly,29000.0,348000.0
4,Yearly,90000.0,90000.0


In [57]:
df['NormalizedAnnualCompensation'].describe()

count    1.058900e+04
mean     6.170771e+06
std      9.842866e+07
min      0.000000e+00
25%      5.200000e+04
50%      1.000000e+05
75%      3.600000e+05
max      8.400000e+09
Name: NormalizedAnnualCompensation, dtype: float64

In [58]:
df.to_csv('data/wrangling_processed_data.csv', index = False)

In [59]:
df_test = pd.read_csv('data/wrangling_processed_data.csv')
df_test.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase,NormalizedAnnualCompensation
0,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy,61000.0
1,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,NaN,23.0,Man,No,Bisexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult,138000.0
2,13,I am a developer by profession,Yes,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Tech articles written by other developers;Cour...,28.0,Man,No,Straight / Heterosexual,White or of European descent,Yes,Appropriate in length,Easy,90000.0
3,16,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,Employed full-time,United Kingdom,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",NaN,...,Tech articles written by other developers;Indu...,26.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult,348000.0
4,17,I am a developer by profession,Yes,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,Employed full-time,Australia,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Tech articles written by other developers;Indu...,29.0,Man,No,Straight / Heterosexual,Hispanic or Latino/Latina;Multiracial,No,Appropriate in length,Easy,90000.0
